# なでしこリーグ試合日程のスクレイピング

## 日程・結果のページからデータを抽出しiCalendar形式のファイルに出力する。


### 日程・結果のページのURL

In [259]:
urlBase = 'http://www.nadeshikoleague.jp/2015/'
urlList = [
    'nadeshiko1/match/index.php','nadeshiko1/match/index2.php'
    ,'nadeshiko2/match/index.php','nadeshiko2/match/index2.php','nadeshiko2/match/index3.php'
    ,'east/match/index.php','east/match/index2.php'
    ,'west/match/index.php','west/match/index2.php'
]
print(urlList)

['nadeshiko1/match/index.php', 'nadeshiko1/match/index2.php', 'nadeshiko2/match/index.php', 'nadeshiko2/match/index2.php', 'nadeshiko2/match/index3.php', 'east/match/index.php', 'east/match/index2.php', 'west/match/index.php', 'west/match/index2.php']


In [313]:
import urllib2
from bs4 import BeautifulSoup
import json

from datetime import datetime
from datetime import timedelta
import uuid

# 省略名を変換する辞書
teamNames = {
    'NORD':'ノルディーア',
    '大和S':'シルフィード',
    '日テレ':'ベレーザ',
    '浦和':'浦和レッズ',
    'I神戸':'レオネッサ',
    '新潟L':'アルビレックス',
    'AS埼玉':'エルフェン',
    'ノジマ':'ノジマステラ',
    'Ａハリマ':'ハリマアルビオン',
    'アンジュ':'アンジュビオレ',
    '福岡AN':'アンクラス',
    'S世田谷':'スフィーダ'
}

def covertString(navStr):
    # NavigableStringを変換する
    return unicode(navStr).encode('utf-8')

def writeEventField(fp, key, val):
    fp.write('{0}:{1}\n'.format(key, val))

def convertTeamName(name):
    if name in teamNames:
        return teamNames[name]
    return name
    
def outputEvent(fp, data):
    writeEventField(fp, 'BEGIN','VEVENT')
    writeEventField(fp, 'UID:',uuid.uuid1().hex)
    
    dt = datetime.now()
    dtstr = dt.strftime("%Y%m%dT%H%M%SZ")
    writeEventField(fp, 'CREATED', dtstr)
    writeEventField(fp, 'DTSTAMP', dtstr)
    
    writeEventField(fp, 'SEQUENCE','10')
    writeEventField(fp, 'TRANSP','OPAQUE')
    
    if 'date' in data:
        s = data['date']
        dt = datetime.strptime(s[:5]+s[-6:],'%m/%d %H:%M')
        dt = dt.replace(year = 2015)
        
    dts = dt.strftime('%Y%m%dT%H%M%S')
    writeEventField(fp, 'DTSTART;TZID=Asia/Tokyo', dts)
    
    dt = dt + timedelta(0,60*110)
    dte = dt.strftime('%Y%m%dT%H%M%S')
    writeEventField(fp, 'DTEND;TZID=Asia/Tokyo', dte)
    
    summary = "{0} {1} {2} {3}".format(data['gameNo'], convertTeamName(data['left']), data['result'], convertTeamName(data['right']))
    writeEventField(fp, 'SUMMARY', summary)
    writeEventField(fp, 'LOCATION', data.get('location','---'))
    # writeEventField(fp, 'DESCRIPTION', data.get('date','---'))
    
    if 'url' in data:
        writeEventField(fp, 'URL;VALUE=URI', data['url'])
    writeEventField(fp, 'END','VEVENT')

def outputICalHeader(fp):
    fp.write('BEGIN:VCALENDAR\n')
    fp.write('VERSION:2.0\n')
    fp.write('METHOD:PUBLISH\n')
    fp.write('PRODID:-PYTHON_SCRAPER\n')
    fp.write("X-WR-CALNAME:Nadeshiko\n")
    fp.write('X-WR-TIMEZONE:Asia/Tokyo\n')
    fp.write('CALSCALE:GREGORIAN\n')
    
    vtimezone = '''BEGIN:VTIMEZONE
TZID:Asia/Tokyo   
BEGIN:DAYLIGHT
TZOFFSETFROM:+0900
DTSTART:19500507T020000
TZNAME:JDT
TZOFFSETTO:+1000
END:DAYLIGHT        
BEGIN:STANDARD
TZOFFSETFROM:+1000
DTSTART:19510908T020000
TZNAME:JST
TZOFFSETTO:+0900
END:STANDARD
END:VTIMEZONE
'''
    fp.write(vtimezone)

def outputToICalendar(dataList):
    fp = open('schedule_nadeshiko.ics','w')

    outputICalHeader(fp)
    
    for game in dataList:
        outputEvent(fp, game)
        
    fp.write('END:VCALENDAR')
    fp.close()
    
def scrapeFromUrl(url):
    html = urllib2.urlopen(url).read()
    soup = BeautifulSoup(html)

    dataList = []
    game = {}
    
    div = soup.find(id='gameSchedule')
    for e in div.children:
        if isinstance(e, Tag):
            if e.name == 'h3':
                # 第N節
                gameNo = covertString(e.string)
                game['gameNo'] = gameNo
            elif e.name == 'table':
                for tr in e.find_all('tr', recursive=False):
                    for td in tr.find_all('td'):
                        classList = td.get('class')
                        if classList is None:
                            # 会場
                            game['location'] = covertString(td.string)
                        elif classList[0] == 'result':
                            # スコア
                            game['result'] = covertString(td.string)
                        elif classList[0] == 'date':
                            # 日付
                            game['date'] = covertString(td.string)
                        elif classList[0] == 'record':
                            # 公式記録
                            record = td.find('a')
                            if record is not None:
                                s = record.get('onclick')
                                l = s.split("'")
                                game['url'] = l[1]
                        elif classList[0] == 'teamName':
                            # チーム名 'left', 'right'
                            game[classList[1]] = covertString(td.string)
                    dataList.append(game)

                    game = {}
                    game['gameNo'] = gameNo
    return dataList

if __name__ == '__main__':
    dataList = []
    for u in urlList:
        print(urlBase +u)
        dataList.extend(scrapeFromUrl(urlBase+u))

    outputToICalendar(dataList) # iCalendarに出力

    print(len(dataList))

http://www.nadeshikoleague.jp/2015/nadeshiko1/match/index.php
http://www.nadeshikoleague.jp/2015/nadeshiko1/match/index2.php
http://www.nadeshikoleague.jp/2015/nadeshiko2/match/index.php
http://www.nadeshikoleague.jp/2015/nadeshiko2/match/index2.php
http://www.nadeshikoleague.jp/2015/nadeshiko2/match/index3.php
http://www.nadeshikoleague.jp/2015/east/match/index.php
http://www.nadeshikoleague.jp/2015/east/match/index2.php
http://www.nadeshikoleague.jp/2015/west/match/index.php
http://www.nadeshikoleague.jp/2015/west/match/index2.php
315
